In [1]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(usecwd=True))

BASE_URL = "https://gigachat.devices.sberbank.ru/api/v1"
#base_gg = "https://router.huggingface.co/v1"
#"http://a6k2.dgx:34000/v1"

BASE_URL = "https://router.huggingface.co/v1"

#"qwen3-32b"
# gigachat/GigaChat
MODEL_NAME = os.getenv("MODEL_NAME", 'GigaChat')



MODEL_NAME_HF ='meta-llama/Llama-3.3-70B-Instruct'
MODEL_NAME_HF ='Qwen/Qwen3-32B'

MODEL_TEMPERATURE_HF =1

os.environ["OPENAI_API_KEY"] = os.getenv("GIGACHAT_AUTH")

In [2]:
os.getenv("GIGACHAT_AUTH")

'MDE5YmM2MjctOWU5ZS03OWE0LWE0OGYtY2FhMTU4MzYxZDgzOjg2MjAwMDgzLThkNzMtNGU0ZS1hNzIxLTM3YWY5ZWY4ZmZlNg=='

In [3]:
from langchain_community.document_loaders import PyMuPDFLoader
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser, BaseOutputParser
from textwrap import dedent

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
import httpx
import json
import requests
import pandas as pd
import numpy as np
import re

from langchain_pymupdf4llm import PyMuPDF4LLMLoader, PyMuPDF4LLMParser
import pymupdf4llm

from trialmind.pubmed import pmid2papers, PubmedAPIWrapper, pmid2biocxml, parse_bioc_xml
from trialmind.api import StudyCharacteristicsExtraction, ScreeningCriteriaGeneration
from trialmind.retrievers import split_text_into_chunks


import time
import extract
%load_ext autoreload
%autoreload 2


### clinical trials

In [3]:
file_path = "docs/CC_215_L00_DL_fusion_fixed_oncobox_ru.pdf"
fin_condition, treatements_eng = extract.info_from_doc(file_path)
fin_condition, treatements_eng

('Colorectal cancer',
 ['Crizotinib',
  'Lorlatinib',
  'Necitumumab',
  'Nimotuzumab',
  'Panitumumab',
  'Cetuximab',
  'Brigatinib',
  'Ramucirumab',
  'Pomalidomide',
  'Toremifene',
  'Denosumab',
  'Duvelisib',
  'Inavolitib',
  'Bevacizumab',
  'Anastrozole',
  'Exemestane'])

1. загрузить док
2. взять 1-ую страницу; рег.выражениями взять диагноз
-- с ЛЛМ перевести на англ.
3. найти страницу с таблицей; или взять таблицы с PyMuPDFLoader
4. извлечь таблицу

In [63]:
' OR '.join(treatements_eng)

'Crizotinib OR Lorlatinib OR Necitumumab OR Nimotuzumab OR Panitumumab OR Cetuximab OR Brigatinib OR Ramucirumab OR Pomalidomide OR Toremifene OR Denosumab OR Duvelisib OR Inavolitib OR Bevacizumab OR Anastrozole OR Exemestane'

In [5]:
all_studies = extract.get_clinicaltrials(fin_condition, 
                                      #' OR '.join(treatements_eng), 
                                       treatements_eng[0],  
                                      max_studies=100)

In [6]:
all_studies

,results,id,title,status,conditions,study_type,phases,interventions,outcomes
0,False,NCT02465060,Targeted Therapy Directed by Genetic Testing i...,ACTIVE_NOT_RECRUITING,"[Advanced Lymphoma, Advanced Malignant Solid N...",INTERVENTIONAL,[PHASE2],"[adavosertib, afatinib, afatinib dimaleate, bi...",NaN
1,False,NCT03947385,Study of IDE196 in Patients With Solid Tumors ...,RECRUITING,"[Metastatic Uveal Melanoma, Cutaneous Melanoma...",INTERVENTIONAL,"[PHASE1, PHASE2]","[ide196, binimetinib, crizotinib]",NaN
2,False,NCT02568267,Basket Study of Entrectinib (RXDX-101) for the...,ACTIVE_NOT_RECRUITING,"[Breast Cancer, Cholangiocarcinoma, Colorectal...",INTERVENTIONAL,[PHASE2],[entrectinib],NaN
3,False,NCT06214793,Taletrectinib in Previously Treated Metastatic...,SUSPENDED,"[Breast Cancer, Metastatic Breast Cancer]",INTERVENTIONAL,[PHASE2],[taletrectinib],NaN
4,False,NCT03792568,ALK Inhibitor in Metastatic Colorectal Cancer ...,UNKNOWN,[Colorectal Cancer],INTERVENTIONAL,[NA],[],NaN
5,False,NCT05725200,Study to Investigate Outcome of Individualized...,RECRUITING,[Metastatic Colorectal Cancer],INTERVENTIONAL,[PHASE2],"[alectinib, cetuximab, crizotinib, dasatinib, ...",NaN
6,True,NCT02510001,MEK and MET Inhibition in Colorectal Cancer,COMPLETED,"[Solid Tumor, Colorectal Cancer]",INTERVENTIONAL,[PHASE1],"[pf-02341066, pd-0325901, binimetinib]","[{'type': 'PRIMARY', 'title': 'Maximal Tolerat..."


### outcome results
Взяли все результаты, отдельно сохранили испытания с результатами, и з них отбираем рез-ты только с unit==participants

filter by interventional study type?

In [251]:
outcome_participants = [obj for item_main in results_studies for obj in item_main['resultsSection']['outcomeMeasuresModule'
                                           ]['outcomeMeasures'
                                            ] if(obj.get('unitOfMeasure','') == 'Participants')] 
outcome_participants

[{'type': 'PRIMARY',
  'title': 'Clinical Response to Binimetinib Combined With PF-02341066',
  'description': 'To investigate response to treatment with RPII dose of Binimetinib with Crizotinib (PF-02341066), in patients with a) RASMT CRC or b) RASWT/cMET mut amplified CRC or c) RASWT/c-MET over-expressed CRC, as defined by stable, partially or completely responding disease, per Response Evaluation Criteria In Solid Tumors Criteria (RECIST v1.1) for target lesions and assessed by CT: Complete Response (CR), Disappearance of all target lesions; Partial Response (PR), \\>=30% decrease in the sum of the longest diameter of target lesions; Stable Disease (SD), Neither sufficient shrinkage to qualify for PR nor sufficient increase (\\>=20%) to qualify for Progressive Disease; Overall Response (OR) = CR + PR + SD',
  'populationDescription': 'Evaluable patients for the primary outcome are those patients who complete a response assessment after cycle 1 of treatment, or who progress early on 

In [273]:
prompt2 = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful medical assistant. Extract information about the efficiency statistics from each outcome measure. Give a concise description with percentages /no_think"),
    ("human", "{text}"),
    # ("ai", "Good.")
])

chain2 = prompt2 | llm_reasoning
response_res = await chain2.ainvoke({"text": outcome_participants[0]})
print(response_res.content)

<think>

</think>

Here is a concise summary of the efficiency statistics for the clinical response to Binimetinib combined with PF-02341066 (Crizotinib) in the Dose Expansion Phase:

- **Total Evaluable Patients**: 30 participants.
- **Stable Disease (SD)**: 7 patients (23.3%).
- **Progressive Disease (PD)**: 22 patients (73.3%).
- **Early Death from Malignant Disease**: 1 patient (3.3%).

These outcomes were assessed using RECIST v1.1 criteria after cycle 1 of treatment.


### results in a given format?

In [ ]:

class Info(BaseModel):
    diagnose: str = Field(..., description="База клинических испытаний")
    treatments: str = Field(..., description="Высшие позиции рейтинга таргетных препаратов")

parser = PydanticOutputParser(pydantic_object=Info)

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    (
        "system", 
        dedent(
            """
            You are a helpful assistant. 
            Your goal is to extract related information from the text provided by the user.
            Return the information in the following JSON-format.
            {fmt}
            """
        )
    ),
    (
        "human", 
        dedent(
            """
            Text: {text}"
            """
        )
    ),
])
chain = (
    prompt 
    | llm_reasoning 
    | parser
)


response = await chain.ainvoke({"text": text, "fmt": parser.get_format_instructions()})
print(response)

### papers

In [5]:
fin_condition = 'Colorectal cancer'
treatements_eng =  ['Crizotinib','Lorlatinib',]

In [6]:
search_api = PubmedAPIWrapper()
# page_size is the max number of records to return!!!! not pages!
tmp_inputs = {
        "page_size": 5,
        "keyword_map": {'conditions':[fin_condition], 
                        'treatments':[treatements_eng[0]]
                       },
        "keywords": {
            "OPERATOR": 'AND'
        }
}
pubmed_api_key= '6892c4129cef143ff92d11533848d2e0d908'#os.getenv("PUBMED_API_KEY", '')
time.sleep(.5)
response = search_api.build_search_query_and_get_pmid(tmp_inputs, 
                                                      api_key=pubmed_api_key)
response

(['40526090', '40211189', '41268440', '41213063', '40369167'],
 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=(Colorectal cancer)+AND+(Crizotinib)&retmax=5&retmode=json&api_key=6892c4129cef143ff92d11533848d2e0d908',
 '67')

In [7]:
df_papers = pmid2papers(pmid_list=response[0], api_key=pubmed_api_key)
df_papers[0].head()

,PMID,Journal,Year,Month,Day,Title,Publication Type,Authors,Abstract
0,40526090,Oncotarget,2025,Jun,17,Case Report WIN-MTB-2023001 WIN International ...,"Journal Article, Case Reports","Alberto Hernando-Calvo, Razelle Kurzrock, Nadi...",Heavily pretreated metastatic colorectal cance...
1,40211189,BMC cancer,2025,Apr,10,A Phase Ia/b study of MEK1/2 inhibitor binimet...,"Journal Article, Clinical Trial, Phase I","Francesca Aroldi, Elena Elez, Thierry André, G...",Targeting RAS mutant (MT) colorectal cancer (C...
2,41268440,Frontiers in pharmacology,2025,,,Case Report: HGF and NF1 mutations as putative...,"Case Reports, Journal Article","Zhitao Chen, Shan Luo, Yangjun Gu, Qiyong Li",Hepatocellular carcinoma (HCC) is a highly agg...
3,41213063,"Journal of environmental pathology, toxicology...",2025,,,Radiotherapy-Related Autophagy Genes Predict P...,Journal Article,"Mao Mao, Yichao Zhuang, Yan Chen",Numerous genes have been associated with color...
4,40369167,Cell death and differentiation,2025,Nov,,ARID1A loss enhances sensitivity to c-MET inhi...,Journal Article,"Xu Zhang, Zihuan Wang, Yilin He, Kejin Wang, C...","ARID1A, a subunit of the SWI/SNF chromatin-rem..."


In [19]:
fin_condition, treatements_eng

('Colorectal cancer', ['Crizotinib', 'Lorlatinib'])

### extract results

In [43]:
fin_condition = 'Colorectal cancer'
treatements_eng =  ['Crizotinib','Lorlatinib',]

In [8]:
os.getenv("GIGACHAT_AUTH")

'MDE5YmM2MjctOWU5ZS03OWE0LWE0OGYtY2FhMTU4MzYxZDgzOjg2MjAwMDgzLThkNzMtNGU0ZS1hNzIxLTM3YWY5ZWY4ZmZlNg=='

In [9]:
import requests
import uuid

url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"
# Создадим идентификатор UUID (36 знаков)
rq_uid = str(uuid.uuid4())
payload={
  'scope': 'GIGACHAT_API_PERS'
}
headers = {
  'Content-Type': 'application/x-www-form-urlencoded',
  'Accept': 'application/json',
  'RqUID': rq_uid,
  'Authorization': f'Basic {os.getenv("GIGACHAT_AUTH")}'
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)
os.environ["OPENAI_API_KEY"] = response.json()['access_token']

os.getenv("OPENAI_API_KEY")

D:\Programs\Anaconda\envs\llms\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ngw.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


'eyJjdHkiOiJqd3QiLCJlbmMiOiJBMjU2Q0JDLUhTNTEyIiwiYWxnIjoiUlNBLU9BRVAtMjU2In0.n3a2FYpWxt9QJhsECw6ggo87K1pT9KrfqJ7AXHi08-i6xR2hFi0noT0lhY3eQ7Nzhv9CskuKrbJhI5y6BiVQEbIJNvWZf6HuhtQp2TWiTlpXe_UC8tRxAhKzJQpkidmHujuKQSRnRugLUgjZZD6YcQaP0wxXeJR4Ae-FGrOrugF5c0oTUi-aXudxHwXDq0h_uyGIHVvz66wnrxVcsE128J8LH0Np7z7TS7P-j0_w3MKZ_7GIOi88lK6gBWNGE2wYivoKI8GB_3c65EC6ItZzrOCqhevWpqBTINAXaRDuXf-EdQG5DFkbqpx4Lse4OwB6XKAsGmwZ1PnLXlrQApgGMw.Eh3pK_ugzcJVwbx7AsWZCw.SnIVvXLeC2FDo2Gf902HrGKrQfYYgsT5-iUKikrIspRAnI3LJcYDaIUpMT_lYK8_W1m8zUtWAzmENmNCVu6hGJyIaAGTuzqDvsgBX0CT7gULHy9mNmEsMqNDsgFtTATdsuYdoB0ZKptb5HoOwlTT-NkcKyWJhsPka_lYe4C5h-4jz7wH6atfjHt5eSuX9ESSWpkREwF2i9tmnRoCVyVUNjr1zune6Qw1FVV4c8LAF6ySJ6sg9CfGktABmRBgCmxbadUGu5CLxuFfhaZZ3CVg73V90VkEb1uqkmnjeA_hijDuX9f1bFt_TWh7ChLileK2WB_W_8JgtM0VfLU6xqs5W329hZdWO17Ctm4diEIH8xe-vgCjnoknsxcKqL4lMTYslscvwLgkQUW8uW5wbCayM0l2YNmW6Z0trwHermNSA5nbRX9v_rPfRAHyJpPsEA5axNfygvoSFTagCuZwRr5lUqjMAI_LTRF2kwG69pm48ULvUZ9RrJ3i4JMqkyTUvW-fzIUebe-h-qurkYBRz7fmT2bDgR4TAB-nqZ2Xa91QyoDo9

In [10]:
from openai import OpenAI

In [11]:
os.environ["BASE_URL"] = "https://gigachat.devices.sberbank.ru/api/v1"
os.environ["MODEL_NAME"] ='GigaChat'

In [12]:
openai_client = OpenAI(
    base_url=os.getenv("BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    http_client=httpx.Client(http2=True, verify=False)
)

response = openai_client.chat.completions.create(
        model=os.getenv("MODEL_NAME"),
        messages=[{"role": "user", "content": "You are a helpful assistant. Create a short poem"}],  # Ensure messages is a list
        temperature=0
    )
response

ChatCompletion(id=None, choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='**Тропинка вдаль бежит,**\n**Лес зелёный шумит,**\n**Солнце светит сквозь листву —**\n**Жизнь прекрасна, вот она!**\n\n**Птицы звонко напевают,**\n**Речка тихо журчит,**\n**И душа свободна вновь,**\n**Сердцу мир дарит миг.**', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1770302252, model='GigaChat:2.0.28.2', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=76, prompt_tokens=20, total_tokens=96, completion_tokens_details=None, prompt_tokens_details=None, precached_prompt_tokens=2))

In [13]:
os.getenv("BASE_URL")

'https://gigachat.devices.sberbank.ru/api/v1'

In [14]:
# test the apis

api = ScreeningCriteriaGeneration()
ecs = api.run(
    population = f"Patients with {fin_condition} undergoing treatment with {treatements_eng[0]}",
    intervention = f"{treatements_eng[0]}",
    comparator = "",
    outcome = "",
    llm = os.getenv("MODEL_NAME"),
    num_title_criteria=3,
    num_abstract_criteria=3,
    thinking=False,
)
ecs

https://gigachat.devices.sberbank.ru/api/v1
https://gigachat.devices.sberbank.ru/api/v1/
{
  "ELIGIBILITY_ANALYSIS": [
    // Eligibility Criteria Rationales
    "Patients must have colorectal cancer — Studies focusing on other types of cancers would not provide relevant data.",
    "Studies involving patients receiving crizotinib as an intervention — Exclusion of studies using different treatments ensures consistency in comparison.",
    "Comparative studies including a control group receiving standard therapy or placebo — Comparisons against standard care or placebo ensure meaningful outcomes.",
    "Outcomes related to survival rates, response rates, toxicity profiles, or quality-of-life measures — These outcomes directly relate to patient management and treatment effectiveness.",
    "Published studies from peer-reviewed journals — Ensures reliability and validity of findings."
  ],
  "TITLE_CRITERIA": [
    // Binary Title-Based Questions
    "Does the title mention 'colorectal ca

{'title_criteria': [], 'content_criteria': [], 'eligibility_analysis': []}

In [14]:
from trialmind.api import LiteratureScreening
api = LiteratureScreening()

papers = df_papers[0].iloc[:3]["Title"] + ": " + df_papers[0].iloc[:3]["Abstract"].fillna("") # important to fillna
papers = papers.tolist()
papers

['Case Report WIN-MTB-2023001 WIN International Molecular Tumor Board A 62-year-old male with metastatic colorectal cancer with 5 prior lines of treatment.: Heavily pretreated metastatic colorectal cancer (mCRC) poses significant therapeutic challenges. Advances in molecular profiling enables personalized strategies. We present a 62-year-old male with mCRC harboring',
 'A Phase Ia/b study of MEK1/2 inhibitor binimetinib with MET inhibitor crizotinib in patients with RAS mutant advanced colorectal cancer (MErCuRIC).: Targeting RAS mutant (MT) colorectal cancer (CRC) remains a difficult challenge, mainly due to the pervasiveness of RAS/MEK-mediated feedback loops. Preclinical studies identified MET/STAT3 as an important mediator of resistance to KRAS-MEK1/2 blockade in RASMT CRC. This dose escalation/expansion study assessed safety and initial efficacy of the MEK1/2 inhibitor binimetinib with MET inhibitor crizotinib in RASMT advanced CRC patients.\nIn the dose escalation phase, patients

In [15]:
title_criteria = ["Does the title mention 'colorectal cancer'?",
"Does the title mention 'crizotinib'?",
"Is there any indication in the title that the study compares crizotinib to another treatment?"
]

content_criteria = ["Are the methods section and results clearly described in the paper?",
"Does the study specify the type of colorectal cancer (e.g., adenocarcinoma)?",
"Were the outcomes measured over a sufficient duration to assess long-term effects?"
  ]

In [19]:
ec_pred = api.run(
    population = f"Patients with {fin_condition} undergoing treatment with {treatements_eng[0]}",
    intervention = f"{treatements_eng[0]}",
    comparator = "",
    outcome = "",
    llm = os.getenv("MODEL_NAME"),
    criteria = title_criteria + content_criteria,
    papers = papers[:3], # make for the top-100 for demo
)
ec_pred

https://gigachat.devices.sberbank.ru/api/v1/
https://gigachat.devices.sberbank.ru/api/v1/
https://gigachat.devices.sberbank.ru/api/v1/


RetryError: RetryError[<Future at 0x27c8ee3a200 state=finished raised RateLimitError>]

In [9]:
pmid_list = df_papers[0].PMID.values.tolist()
#['41213063',#'26451310',]

res = pmid2biocxml(pmid_list, api_key=pubmed_api_key)
#print(res)
res = [parse_bioc_xml(r) for r in res]
# transform the parsed xml into paper content
papers = []
for parsed in res:
    paper_content = []
    for parsed_ in parsed["passage"]:
        paper_content.append(parsed_['content'])
    paper_content = "\n".join(paper_content)
    papers.append(paper_content)


https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/41268440/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908
https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/41213063/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908
https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/40526090/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908
https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/40211189/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908
https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/40369167/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908


In [11]:
[len(i) for i in papers]

[1672, 1652, 27171, 2627, 46819]

In [15]:
splited = split_text_into_chunks(papers[1], chunk_size=600,
                                             chunk_overlap=300)
splited

['Case Report: HGF and NF1 mutations as putative bypass mechanisms of MET inhibitor resistance in hepatocellular carcinoma: a case study.',
 'Background: Hepatocellular carcinoma (HCC) is a highly aggressive liver cancer with poor prognosis, often associated with resistance to treatment. MET amplification has been identified as a potential therapeutic target, but resistance to MET inhibitors, such as crizotinib, remains a significant challenge. This study aims to explore the molecular mechanisms underlying resistance to MET inhibitors in MET-amplified HCC. Methods: We present a case of advanced HCC in a patient with MET amplification treated with crizotinib. After initial tumor regression, disease progression occurred. Genetic',
 'challenge. This study aims to explore the molecular mechanisms underlying resistance to MET inhibitors in MET-amplified HCC. Methods: We present a case of advanced HCC in a patient with MET amplification treated with crizotinib. After initial tumor regression

In [12]:

api = StudyCharacteristicsExtraction()
extracted = api.run(
    papers=papers,
    fields=['Results, string, main results of the study regarding Crizotinib, usually about treatment effectiveness'],#api.DEFAULT_FIELDS,
    llm='Qwen/Qwen3-32B',#"openai-gpt-4o",
    chunk_size=600,
    chunk_overlap=300,
    thinking=False,
)
extracted

False


[[{'name': 'Crizotinib treatment effectiveness',
   'value': 'initial tumor regression followed by disease progression',
   'source_id': [2],
   'cited_blocks': ['challenge. This study aims to explore the molecular mechanisms underlying resistance to MET inhibitors in MET-amplified HCC. Methods: We present a case of advanced HCC in a patient with MET amplification treated with crizotinib. After initial tumor regression, disease progression occurred. Genetic analysis using next-generation sequencing (NGS) was performed on biopsy samples taken before and after progression to identify mutations associated with resistance. Results: NGS revealed the loss of MET amplification and identified HGF and NF1 mutations as potential bypass mechanisms. Specifically,']}],
 [{'name': 'Crizotinib effectiveness',
   'value': 'More sensitive to low-risk group',
   'source_id': [5],
   'cited_blocks': ['performed in risk groups. Expression of signature genes in CRC cells was examined using quantitative rev

[[{'name': 'Crizotinib effectiveness',
   'value': 'More sensitive to low-risk group',
   'source_id': [4],
   'cited_blocks': ['performed in risk groups. Expression of signature genes in CRC cells was examined using quantitative reverse transcription polymerase chain reaction (qRT-PCR). There were 10 CRC-related RRAGs found. Prognostic models were built with RRAGs. Based on immune infiltration analysis, low-risk populations showed significantly greater levels of immune infiltration (P P ARHGEF17 mutation rate was 6%. Medications such as CGP-082996, Dasatinib, Erlotinib, and Salubrinal were more sensitive to high-risk group, whereas drugs such as FTI-277, DMOG, and Crizotinib were more sensitive to low-risk group.']}],
 [{'name': 'Crizotinib treatment effectiveness',
   'value': 'initial tumor regression followed by disease progression',
   'source_id': [1],
   'cited_blocks': ['challenge. This study aims to explore the molecular mechanisms underlying resistance to MET inhibitors in ME

In [38]:
llm_reasoning = ChatOpenAI(
    base_url=BASE_URL,
    api_key=HUGGINGFACE_HUB_TOKEN,
    model=MODEL_NAME_HF,
    streaming=True,
    temperature=0,
    #http_client=httpx_client
    http_async_client=httpx.AsyncClient(verify=False)
)

In [47]:
prompt = ChatPromptTemplate.from_messages([
    ("user", '/no_think {text}'),
    # ("ai", "Good.")
])

chain = prompt | llm_reasoning
response = await chain.ainvoke({"text": ff})

#fin_condition = response.content.strip('<think>\n\n</think>\n\n')
print(response.content)

<think>

</think>

```json
[
    {
        "name": "Crizotinib effectiveness",
        "value": "More sensitive to low-risk group",
        "source_id": [3]
    }
]
```
